In [4]:
import pandas as pd
import pickle
from geopy.distance import geodesic
import numpy as np

# Cargar los modelos y los datos
df_tiendas = pickle.load(open("tiendas.pkl", "rb"))
cosine_sim = pickle.load(open("tfidf_similarity.pkl", "rb"))

def obtener_recomendaciones_con_distancia(tienda_id, user_location, cosine_sim=cosine_sim, radius_km=1):
    if tienda_id not in df_tiendas['tienda_id'].values:
        return []  # O puedes devolver un mensaje de error adecuado
    
    idx = df_tiendas.index[df_tiendas['tienda_id'] == tienda_id].tolist()[0]
    
    # Prefiltrar tiendas dentro de un cuadrado de lado 2 * radius_km
    lat_min = user_location[0] - radius_km / 111  # Aproximación: 1 grado de latitud ≈ 111 km
    lat_max = user_location[0] + radius_km / 111
    lon_min = user_location[1] - radius_km / (111 * np.cos(np.radians(user_location[0])))
    lon_max = user_location[1] + radius_km / (111 * np.cos(np.radians(user_location[0])))
    
    prefiltered_tiendas = df_tiendas[
        (df_tiendas['latitud'] >= lat_min) & (df_tiendas['latitud'] <= lat_max) &
        (df_tiendas['longitud'] >= lon_min) & (df_tiendas['longitud'] <= lon_max)
    ]
    
    # Calcular distancias geodésicas solo para las tiendas prefiltradas
    tiendas_dentro_radio = []
    for i in prefiltered_tiendas.index:
        tienda_location = (prefiltered_tiendas.at[i, 'latitud'], prefiltered_tiendas.at[i, 'longitud'])
        distance = geodesic(user_location, tienda_location).km
        if distance <= radius_km:
            tiendas_dentro_radio.append((i, distance))
    
    if not tiendas_dentro_radio:
        return []  # No hay tiendas dentro del radio especificado
    
    # Calcular similitud del coseno solo para las tiendas dentro del radio
    sim_scores = [(i, cosine_sim[idx][i]) for i, _ in tiendas_dentro_radio]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:4]  # Obtener los 4 más similares
    tienda_indices = [i[0] for i in sim_scores]
    
    recomendaciones = []
    for i in tienda_indices:
        tienda_location = (df_tiendas.at[i, 'latitud'], df_tiendas.at[i, 'longitud'])
        distance = geodesic(user_location, tienda_location).km
        tienda_info = df_tiendas.iloc[i].to_dict()
        tienda_info['distance'] = round(distance, 2)
        recomendaciones.append(tienda_info)
        print(f"Tienda: {df_tiendas.at[i, 'nombre']}, Distancia: {distance:.2f} km, Similitud: {sim_scores[tienda_indices.index(i)][1]:.2f}")  # Imprimir distancia y similitud
    
    return recomendaciones


# 19321: pardos
# Uso de ejemplo
#9143: La Lucha Sangucheria Criolla
# -12.135105794453583, -77.02212741858906 coordenadas UTEC
#17989: Consultorio Odontologico Brasilerio
#user_location = (-12.130843, -76.982489)  # Ubicación del usuario (para probar puse ubicacion de la tienda)
tienda_id = 17989  # ID de la tienda
user_location = (-12.135105794453583, -77.02212741858906)  # Ubicación del usuario (para probar puse ubicacion de la tienda)
radius_km = 1  # Radio de búsqueda en kilómetros
print(f'Tienda: {tienda_id}, Ubicación: {user_location}, Radio: {radius_km}')
recomendaciones = obtener_recomendaciones_con_distancia(tienda_id, user_location, radius_km=radius_km)  # Aumenta el radio para depuración
print(recomendaciones)

Tienda: 17989, Ubicación: (-12.135105794453583, -77.02212741858906), Radio: 1
Tienda: Consultorio Odontologico Brasileiro, Distancia: 0.20 km, Similitud: 1.00
Tienda: APOPS, Distancia: 0.58 km, Similitud: 0.30
Tienda: Drs Mendoza, Distancia: 0.59 km, Similitud: 0.21
Tienda: Sonrise - Clinica Odontologica - Sede Miraflores, Distancia: 0.51 km, Similitud: 0.18
[{'tienda_id': 17989, 'nombre': 'Consultorio Odontologico Brasileiro', 'categorias': 'dentist, health', 'latitud': -12.136867, 'longitud': -77.0224193, 'direccion': 'Avenida Almirante Miguel Grau 1502, Barranco', 'calificaciones': 5.0, 'rango_precios': 'Medio', 'metodos_pago': 'Efectivo', 'tamano_tienda': 'Grande', 'estacionamiento': 'Sí', 'horario_apertura': 'Fines de semana', 'distrito': 'Barranco', 'zona': 'Lima Sur', 'combined_features': 'Consultorio Odontologico Brasileiro dentist, health Medio Efectivo Grande Sí Fines de semana Lima Sur', 'distance': 0.2}, {'tienda_id': 17075, 'nombre': 'APOPS', 'categorias': 'dentist, health